<a href="https://colab.research.google.com/github/AksharaSiddharthan/Womens-Threat-Detection-System/blob/main/women_threat_detection_ai_proj.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from sklearn.tree import export_text, export_graphviz
# textual rules for tree 0
rules = export_text(clf.estimators_[0], feature_names=list(X.columns))
print(rules)

# feature importances
importances = clf.feature_importances_
for f,imp in sorted(zip(X.columns, importances), key=lambda x:-x[1]):
    print(f, imp)


|--- Cognitive_State_enc <= 0.50
|   |--- GSR_Values <= 0.42
|   |   |--- GSR_Values <= 0.32
|   |   |   |--- Age <= 23.50
|   |   |   |   |--- GSR_Values <= 0.19
|   |   |   |   |   |--- GSR_Values <= 0.05
|   |   |   |   |   |   |--- Emotional_State_enc <= 1.00
|   |   |   |   |   |   |   |--- Age <= 22.50
|   |   |   |   |   |   |   |   |--- class: 1.0
|   |   |   |   |   |   |   |--- Age >  22.50
|   |   |   |   |   |   |   |   |--- class: 0.0
|   |   |   |   |   |   |--- Emotional_State_enc >  1.00
|   |   |   |   |   |   |   |--- class: 0.0
|   |   |   |   |   |--- GSR_Values >  0.05
|   |   |   |   |   |   |--- class: 0.0
|   |   |   |   |--- GSR_Values >  0.19
|   |   |   |   |   |--- class: 1.0
|   |   |   |--- Age >  23.50
|   |   |   |   |--- class: 1.0
|   |   |--- GSR_Values >  0.32
|   |   |   |--- class: 0.0
|   |--- GSR_Values >  0.42
|   |   |--- Age <= 23.50
|   |   |   |--- Age <= 22.50
|   |   |   |   |--- GSR_Values <= 1.84
|   |   |   |   |   |--- Emotional_State_

In [ ]:
# VOTINGCLASSIFIER(RANDOM FOREST CLASSIFIER+ Support Vector Machine (SVM) + logic regression)

import pandas as pd
import ast
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier, VotingClassifier
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score
from sklearn.preprocessing import LabelEncoder

# Load dataset
data = pd.read_csv("mental_health_wearable_data.csv")

# Convert list-like strings to numerical averages
for col in data.columns:
    if data[col].dtype == 'object':
        try:
            data[col] = data[col].apply(lambda x: ast.literal_eval(x) if isinstance(x, str) and x.startswith('[') else x)
        except Exception:
            pass

# Flatten list columns (average values)
for col in data.columns:
    if isinstance(data[col].iloc[0], list):
        data[col] = data[col].apply(lambda x: sum(x)/len(x) if isinstance(x, list) else x)

# Encode any remaining categorical columns numerically
for col in data.columns:
    if data[col].dtype == 'object':
        le = LabelEncoder()
        data[col] = le.fit_transform(data[col].astype(str))

# Ensure 'Target' column exists; if not, use the last column as the label
if "Target" in data.columns:
    X = data.drop(columns=["Target"])
    y = data["Target"]
else:
    X = data.iloc[:, :-1]
    y = data.iloc[:, -1]

# Split dataset into 90% train, 10% test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=42, shuffle=True)

# Define base models
rf = RandomForestClassifier(n_estimators=400, random_state=42)
svm = SVC(kernel='rbf', probability=True, random_state=42)
lr = LogisticRegression(max_iter=1000, random_state=42)

# Voting Classifier (soft voting = probability-based)
voting_clf = VotingClassifier(
    estimators=[('rf', rf), ('svm', svm), ('lr', lr)],
    voting='soft'
)

# Train ensemble on 90% data
voting_clf.fit(X_train, y_train)

# Predict on 10% unseen data
y_pred = voting_clf.predict(X_test)

# Evaluation metrics
accuracy = accuracy_score(y_test, y_pred)
cm = confusion_matrix(y_test, y_pred)
report = classification_report(y_test, y_pred)

# Display results
print("VOTING CLASSIFIER")
print(" Model Evaluation on 10% Test Data")
print("-------------------------------------")
print(f"Accuracy: {accuracy:.4f}")
print("\nConfusion Matrix:\n", cm)
print("\nClassification Report:\n", report)

# Save actual vs predicted results
output = pd.DataFrame({"Actual": y_test, "Predicted": y_pred})
output.to_csv("voting_classifier_results.csv", index=False)
print("\n Results saved to 'voting_classifier_results.csv'")


VOTING CLASSIFIER
 Model Evaluation on 10% Test Data
-------------------------------------
Accuracy: 0.4600

Confusion Matrix:
 [[11 14]
 [13 12]]

Classification Report:
               precision    recall  f1-score   support

           0       0.46      0.44      0.45        25
           1       0.46      0.48      0.47        25

    accuracy                           0.46        50
   macro avg       0.46      0.46      0.46        50
weighted avg       0.46      0.46      0.46        50


 Results saved to 'voting_classifier_results.csv'


In [ ]:
# only RANDOM FOREST CLASSIFIER
import pandas as pd
import ast
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score
from sklearn.preprocessing import LabelEncoder

# Load dataset
data = pd.read_csv("mental_health_wearable_data.csv")

# Convert list-like strings to numerical averages
for col in data.columns:
    if data[col].dtype == 'object':
        try:
            data[col] = data[col].apply(lambda x: ast.literal_eval(x) if isinstance(x, str) and x.startswith('[') else x)
        except Exception:
            pass

# Flatten list columns (average values)
for col in data.columns:
    if isinstance(data[col].iloc[0], list):
        data[col] = data[col].apply(lambda x: sum(x)/len(x) if isinstance(x, list) else x)

# Encode categorical columns if any remain
for col in data.columns:
    if data[col].dtype == 'object':
        le = LabelEncoder()
        data[col] = le.fit_transform(data[col].astype(str))

# Separate features and target
if "Target" in data.columns:
    X = data.drop(columns=["Target"])
    y = data["Target"]
else:
    X = data.iloc[:, :-1]
    y = data.iloc[:, -1]

# Split 90% train, 10% test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=42, shuffle=True)

# Random Forest model
rf = RandomForestClassifier(n_estimators=450, random_state=42)


rf.fit(X_train, y_train)

# Predictions
y_pred = rf.predict(X_test)


# Evaluation metrics
accuracy = accuracy_score(y_test, y_pred)
cm = confusion_matrix(y_test, y_pred)
report = classification_report(y_test, y_pred)

# Display results
print(" Random Forest Evaluation (90% Train, 10% Test)")
print("-----------------------------------------------")
print(f"Accuracy: {accuracy:.4f}")
print("\nConfusion Matrix:\n", cm)
# Print confusion matrix with labels
cm = confusion_matrix(y_test, y_pred)
tn, fp, fn, tp = cm.ravel()

print("\nConfusion Matrix (Labeled):")
print(f"True Negatives  (TN): {tn}")
print(f"False Positives (FP): {fp}")
print(f"False Negatives (FN): {fn}")
print(f"True Positives  (TP): {tp}")

print("\n Accuracy:", accuracy)

print("\nClassification Report:\n", report)

# Save predictions
output = pd.DataFrame({"Actual": y_test, "Predicted": y_pred})
output.to_csv("random_forest_results.csv", index=False)
print("\n Results saved to 'random_forest_results.csv'")


 Random Forest Evaluation (90% Train, 10% Test)
-----------------------------------------------
Accuracy: 0.5800

Confusion Matrix:
 [[15 10]
 [11 14]]

Confusion Matrix (Labeled):
True Negatives  (TN): 15
False Positives (FP): 10
False Negatives (FN): 11
True Positives  (TP): 14

 Accuracy: 0.58

Classification Report:
               precision    recall  f1-score   support

           0       0.58      0.60      0.59        25
           1       0.58      0.56      0.57        25

    accuracy                           0.58        50
   macro avg       0.58      0.58      0.58        50
weighted avg       0.58      0.58      0.58        50


 Results saved to 'random_forest_results.csv'


In [ ]:
importances = clf.feature_importances_
for f, imp in sorted(zip(X.columns, importances), key=lambda x: -x[1]):
    print(f, imp)


GSR_Values 0.3132030012041217
EEG_Frequency_Bands 0.3118296869679566
Age 0.3065241683654991


In [ ]:
# MLP

import pandas as pd
import ast
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score

# -------------------------------
# 1️⃣ Load and clean dataset
# -------------------------------
data = pd.read_csv("mental_health_wearable_data.csv")

# Convert list-like strings (e.g., "[1.2, 3.4, 5.6]") to numeric averages
for col in data.columns:
    if data[col].dtype == 'object':
        try:
            data[col] = data[col].apply(lambda x: ast.literal_eval(x) if isinstance(x, str) and x.startswith('[') else x)
        except Exception:
            pass

# Flatten list columns (convert to mean values)
for col in data.columns:
    if isinstance(data[col].iloc[0], list):
        data[col] = data[col].apply(lambda x: sum(x) / len(x) if isinstance(x, list) else x)

# Keep only numeric columns
data = data.select_dtypes(include=['number'])

# -------------------------------
# 2️⃣ Define features and target
# -------------------------------
X = data.drop(columns=["Target"], errors='ignore')
y = data["Target"] if "Target" in data.columns else data.iloc[:, -1]

# -------------------------------
# 3️⃣ Normalize features
# -------------------------------
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# -------------------------------
# 4️⃣ Train-test split (90% / 10%)
# -------------------------------
X_train, X_test, y_train, y_test = train_test_split(
    X_scaled, y, test_size=0.1, stratify=y, random_state=42
)

# -------------------------------
# 5️⃣ Define and train MLP model
# -------------------------------
mlp = MLPClassifier(hidden_layer_sizes=(128, 64), max_iter=500, random_state=42)
mlp.fit(X_train, y_train)

# -------------------------------
# 6️⃣ Predictions and metrics
# -------------------------------
y_pred = mlp.predict(X_test)

accuracy = accuracy_score(y_test, y_pred)
cm = confusion_matrix(y_test, y_pred)
report = classification_report(y_test, y_pred)

# -------------------------------
# 7️⃣ Print results
# -------------------------------
print("\nMLP Neural Network Evaluation (90% Train, 10% Test)")
print("-------------------------------------------------------")
print(f"Accuracy: {accuracy:.4f}")

tn, fp, fn, tp = cm.ravel()
print("\nConfusion Matrix:",cm)
print(f"True Negatives  (TN): {tn}")
print(f"False Positives (FP): {fp}")
print(f"False Negatives (FN): {fn}")
print(f"True Positives  (TP): {tp}")

print("\nClassification Report:\n", report)

# -------------------------------
# 8️⃣ Save predictions
# -------------------------------
output = pd.DataFrame({"Actual": y_test, "Predicted": y_pred})
output.to_csv("mlp_classifier_results.csv", index=False)
print("\nResults saved to 'mlp_classifier_results.csv'")



MLP Neural Network Evaluation (90% Train, 10% Test)
-------------------------------------------------------
Accuracy: 0.5000

Confusion Matrix: [[11 14]
 [11 14]]
True Negatives  (TN): 11
False Positives (FP): 14
False Negatives (FN): 11
True Positives  (TP): 14

Classification Report:
               precision    recall  f1-score   support

           0       0.50      0.44      0.47        25
           1       0.50      0.56      0.53        25

    accuracy                           0.50        50
   macro avg       0.50      0.50      0.50        50
weighted avg       0.50      0.50      0.50        50


Results saved to 'mlp_classifier_results.csv'
